### Сравнение ресторанов грузинской и итальянской кухни в Санкт-Петербурге

**Описание**:
Анализ отзывов, оценок посетителей на рестораны грузинской и итальянской кухни в СПБ 

Данные буду брать с сайта Irecommend


In [1]:
from selenium import webdriver
from selenium_stealth import stealth
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
import pandas as pd
import seaborn as sns                    
import matplotlib.pyplot as plt             
%matplotlib inline  
from wordcloud import WordCloud
from stop_words import get_stop_words
import statistics
import plotly.express as px
from scipy.stats import spearmanr
#!pip install streamlit

# Загрузка данных

В данном разделе выполните выгрузку данных из внешних источников и их локальное сохранение

Вы можете вынести всю логику выгрузки в отдельный `.py` скрипт (или набор скриптов), поместив их в директорию `scripts` в корне проекта, но продублируйте весь код здесь тоже, чтобы можно было запустить ноутбук с самого начала

In [ ]:
def create_web_driver_stealth() -> webdriver.Chrome:
    options = webdriver.ChromeOptions()
    options.add_argument("start-maximized")
    #options.add_argument("--headless")
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_experimental_option('useAutomationExtension', False)

    s = Service(ChromeDriverManager().install())
    stealth_driver = webdriver.Chrome(service=s, options=options)

    stealth(stealth_driver,
            languages=["en-US", "en"],
            vendor="Google Inc.",
            platform="Win32",
            webgl_vendor="Intel Inc.",
            renderer="Intel Iris OpenGL Engine",
            fix_hairline=True,
            )

    return stealth_driver

In [ ]:
def data_collection(base_url):
    driver = create_web_driver_stealth()
    dictionary={}
    url=''
    names = []
    rating = []
    review = []
    likes_list = []
    dislikes_list = []
    for i in range(6):
        if i == 0:
            url = base_url
        else:
            url = f'{base_url}?page={i}'
            
        driver.get(url)
        name_restaurants = driver.find_elements(by=By.CLASS_NAME, value="title")
        rating_restaurants = driver.find_elements(by=By.CLASS_NAME, value="average-rating")
        reviews = driver.find_elements(by=By.CLASS_NAME, value="extract")
    
        for el in rating_restaurants:  
            rating_text = el.text
        
        
        for restaurant in name_restaurants:
            names.append(restaurant.text)
        for restaurant_rating in rating_restaurants:
            rating.append(rating_text)
        for restaurant_review in reviews:
            review.append(restaurant_review.text)
       
        likes = driver.find_elements(By.CLASS_NAME, value="RecommendRating-like")
        for el in likes:
            like_answ = el.text
            likes_list.append(like_answ)
    
        dislikes = driver.find_elements(By.CLASS_NAME, value="RecommendRating-dislike")
        for el in dislikes:
            dislike_answ = el.text
            dislikes_list.append(dislike_answ)
    dictionary["name"] = names
    dictionary["rating"] = rating
    dictionary["reviews"] = review
    dictionary["likes"] = likes_list
    dictionary["dislikes"] = dislikes_list
    return dictionary

In [ ]:
georgian_data = dict()
italian_data = dict()

In [ ]:
georgian_data = data_collection('https://irecommend.ru/catalog/list/9700-9985-23777')

In [ ]:
italian_data = data_collection('https://irecommend.ru/catalog/list/9700-9985-11335')

In [ ]:
df_italian = pd.DataFrame(italian_data)
df_georgian = pd.DataFrame(georgian_data)

In [ ]:
df_italian

In [ ]:
df_georgian

## Очистка данных и подготовка к анализу

В данном разделе выполните все шаги, связанные с предобработкой:
- приведение ваших данных к формату датафрейма (если это не было сделано выше)
- разведочный анализ (EDA)

Exploratory Data Analysis (EDA) - важный этап анализа данных, помогающий понять, с чем мы вообще имеем дело. [Пример выполнения разведанализа](https://www.kaggle.com/code/imoore/intro-to-exploratory-data-analysis-eda-in-python). Не копируйте все функции в лоб - на что именно стоит смотреть зависит от вашего исследования, однако точно стоит посмотреть на:
- какие переменные (колонки) вообще есть
- каких они типов
- есть ли пропуски
- как переменные распределены (если в этом есть смысл). Для по крайней мере начального анализа распределений и/или трендов в данных, возможно, есть смысл построить визуализации

---

Если хотите, можете попробовать вот [эту библиотеку](https://docs.profiling.ydata.ai/latest/) (полезна для EDA), но огромная просьба - не вставляйте куски кода с ней в конечный отчёт, ибо запуска процесса профилирования может быть довольно долгим, а на больших датасетах это может приводить к довольно тяжелым ноутбукам. Если будете использовать вышеупомянутый `ydata-profiling`, делайте это в отдельном ноутбуке (назовите его `EDA.ipynb`), а в этом постарайтесь обойтись без него

In [ ]:
df_italian = df_italian.drop(range(112, 120)) 
# кол-во ресторанов грузинской и итальянской кухни должно быть одинаково, тк я их сравниваю

In [ ]:
df_georgian.shape 

In [ ]:
df_italian.shape 

In [ ]:
df_georgian.dtypes 
# рейтинг сейчас-object, а разумнее, чтобы эта колонка была формата float

In [ ]:
df_italian.dtypes
# рейтинг сейчас-object, а разумнее, чтобы эта колонка была формата float

In [ ]:
df_georgian.isnull().sum() 
# нет неопределенных значений, все поля заполнены информацией

In [ ]:
df_italian.isnull().sum() 
# нет неопределенных значений, все поля заполнены информацией

In [ ]:
df_georgian.describe 

Сейчас колонка 'рейтинг'-строка, поэтому мне следует удалить 'Среднее:' и преобразовать колонку в тип float. 
Также кроме названия в колоне name написано еще Санкт-Петербург/Россия. 
В этом нет смысла, тк я исследую г. Санкт-Петербург, Россия

In [ ]:
df_italian.describe 

Мне не нравится, что повторяется 'Среднее:  и Санкт-Петербург/ Россия(я и так анализирую тольго город СПб), уберу

In [ ]:
df_georgian['rating'] = df_georgian['rating'].apply(lambda x: x.replace('Среднее: ', ''))

In [ ]:
df_georgian['rating'] = pd.to_numeric(df_georgian['rating'])  

In [ ]:
df_georgian['name'] = df_georgian['name'].str.split(',', n=1, expand=True)[0]

In [ ]:
df_georgian

In [ ]:
df_italian['rating'] = df_italian['rating'].apply(lambda x: x.replace('Среднее: ', ''))

In [ ]:
df_italian['rating'] = pd.to_numeric(df_italian['rating']) 

In [ ]:
df_italian['name'] = df_italian['name'].str.split(',', n=1, expand=True)[0]

In [ ]:
df_italian

In [ ]:
df_georgian.dtypes

In [ ]:
df_italian.dtypes

Теперь всё в порядке, можно записывать рестораны в эксель таблицы(сделаю 3 штуки: 
итальянские рестораны, грузинские рестораны и общую таблицу с обеими кухнями)

In [ ]:
df_italian.to_excel("italian_new.xlsx", index=False)

In [ ]:
df_georgian.to_excel("georgian_new.xlsx", index=False)

Структура таблицы:
name-название ресторана
rating-
reviews-отзывы посетителей, только те, которые видны на главной странице с ресторанами
likes-количество лайков за ресторан
dislikes-количество дизлайков за ресторан

Важное замечание: количество лайков+количество дизлайков=количеству отзывов на ресторан

In [ ]:
df_georgian['cuisine'] = 'Грузинская'
df_italian['cuisine'] = 'Итальянская'

In [ ]:
df_restaurants = pd.concat([df_georgian, df_italian], ignore_index=True)

In [ ]:
df_restaurants.to_excel('all_restaurants.xlsx', index=False)

In [ ]:
df_restaurants

## Анализ и визуализация

Весь мыслительный процесс, связанный с аналитикой ваших данных, а также построения визуализаций поместите в данный раздел

In [ ]:
plt.hist(df_georgian['rating'], bins=25, color='blue', alpha=0.8)
plt.xlabel('Оценка')
plt.ylabel('Частота')
plt.title('Распределение оценок грузинских ресторанов')
plt.show()

По гистограмме видно, что больше всего оценок 5.0 у грузинских ресторанов. 
Посетители очень удовлетворены, но также есть и неудовлетворительные оценки, их меньше, но они тоже есть. 
Может быть, это связано с субъективными препочтениями гостей или с долгим ожиданием блюд 

In [ ]:
plt.hist(df_italian['rating'], bins=25, color='blue', alpha=0.8)
plt.xlabel('Оценка')
plt.ylabel('Частота')
plt.title('Распределение оценок итальянских ресторанов')
plt.show()

In [ ]:
Распределение оценок итальянских ресторанов более равномерное, чем у грузинских ресторанов. 
Разница в частоте между кол-вом оценок 5.0 и 2.0-3.5 незначительное. 
Это свидетельствует о разном впечатлении людей на итальянские рестораны 

In [ ]:
statistics.median(df_italian['rating'])

In [ ]:
statistics.median(df_restaurants['rating'])

In [ ]:
statistics.median(df_georgian['rating']) 

In [ ]:
Сравнение медиан рейтинга ресторанов подтверждает вышесказанное. 
Большинство посетителей грузинских ресторанов остаются довольными(медиана равна 4.6, что практически близко к максимальной оценке),
а посетители итальянской кухни не так высоко оценивают рестораны

In [ ]:
sns.boxplot(x='cuisine', y='rating', data=df_restaurants)
plt.title('Сравнение рейтингов ресторанов (Box Plot)')
plt.xlabel('Тип кухни')
plt.ylabel('Рейтинг')
plt.show()

График показывает, что медиана в грузинских ресторанах находится в верхней части boxplot. Большинство грузинских ресторанов имеют высокий рейтинг.
Часть грузинских ресторанов имеет очень низкие рейтинги.
Медиана в итальянских ресторанах находится приблизительно посередине графика. Значит, большинство ресторанов с итальянской кухней имеют средний рейтинг, примерно около оценки 4.0. 
В рейтингах итальянских ресторанов разброс оценок меньше, чем в грузинских.
Разница в оценках грузинской кухни и итальянской может быть вызвана тем, что грузинская зачастую ассоциируется с гостеприимством и большими порциями, а итальянская с изысканностью, высокими требованиями к продуктам

In [ ]:
#на графике - 10 ресторанов грузинской кухни с наивысшим кол-ом лайков
sns.set_style("darkgrid", {'axes.facecolor': '#f0f0f0', 'grid.color': 'white'})
df_georgian = df_georgian.sort_values('likes', ascending=False)
df_georgian_top10 = df_georgian.head(10)
sns.barplot(x='likes', y='name', data=df_georgian_top10, palette='hls', hue='name') 
plt.xlabel('Количество лайков')
plt.ylabel('Ресторан')
plt.title('Топ 10 грузинских ресторанов по количеству лайков')
plt.show()

In [ ]:
#на графике - 10 ресторанов итальянской кухни с наивысшим кол-ом лайков
df_italian = df_italian.sort_values('likes', ascending=False)
df_italian_top10 = df_italian.head(10)
sns.barplot(x='likes', y='name', data=df_italian_top10, palette='hls', hue='name') 
plt.xlabel('Количество лайков')
plt.ylabel('Ресторан')
plt.title('Топ 10 итальянских ресторанов по количеству лайков')
plt.show()

In [ ]:
sns.scatterplot(x='likes', y='rating', data=df_italian)
plt.title('Зависимость между количеством лайков и рейтингом в итальянских ресторанах')
plt.xlabel('Количество лайков')
plt.ylabel('Рейтинг')
plt.show()

По графику видно, что связь нелинейная. Буду использовать корреляцию Спирмена, 
чтобы установить, есть ли связь между кол-вом лайков и рейтингом в итальянских ресторанах

In [ ]:
corr_italian, p_value_italian = spearmanr(df_italian['likes'], df_italian['rating'])

In [ ]:
corr_italian

In [ ]:
p_value_italian

Корреляция равна 0.04210725706337952. Это свидетельствует о том, что связь очень слабая или ее вообще нет. 
P-value равно 0.6593449469433088, то есть может утверждать: количество лайков практически не влияет на рейтинг ресторанов (или наоборот)

In [ ]:
sns.scatterplot(x='likes', y='rating', data=df_georgian)
plt.title('Зависимость между количеством лайков и рейтингом в грузинских ресторанах')
plt.xlabel('Количество лайков')
plt.ylabel('Рейтинг')
plt.show()

In [ ]:
corr_georgian, p_value_georgian = spearmanr(df_georgian['likes'], df_georgian['rating'])

In [ ]:
corr_georgian

In [ ]:
p_value_georgian

По графику видно,что данные распределены неравномерно
Наблюдается слабая отрицательная связь(корреляция равна -0.18051482694993368)
Также связь не является статистически значимой(p_value= 0.056827469403604995,однако 
наблюдаются слабые признаки возможной зависимости, которые могли оказаться случайностью)

In [ ]:
#Проделаем аналогичные операции для установления связи между кол-вом дизлайков и рейтингом
sns.scatterplot(x='dislikes', y='rating', data=df_georgian)
plt.title('Зависимость между количеством дизлайков и рейтингом в грузинских ресторанах')
plt.xlabel('Количество лайков')
plt.ylabel('Рейтинг')
plt.show()

In [ ]:
corr_georgian_dis, p_value_georgian_dis = spearmanr(df_georgian['dislikes'], df_georgian['rating'])

In [ ]:
corr_georgian_dis

In [ ]:
p_value_georgian_dis

In [ ]:
sns.scatterplot(x='dislikes', y='rating', data=df_italian)
plt.title('Зависимость между количеством дизлайков и рейтингом в итальянских ресторанах')
plt.xlabel('Количество лайков')
plt.ylabel('Рейтинг')
plt.show()

На графике видно, что большое кол-во ресторанов имеют 0-25 дизлайков. 
Интересно, что они имеют как низкий, так и высокий рейтинг

In [ ]:
corr_italian_dis, p_value_italian_dis = spearmanr(df_italian['dislikes'], df_italian['rating'])

In [ ]:
corr_italian_dis

In [ ]:
p_value_italian_dis

По результатам двух графиков выше, можно утверждать:данные распределены неравномерно. 
По корреляции и p_value в грузинских и итальянских ресторанах не наблюдается связь между кол-вом дизлайков 
и рейтингом, связь не статистически значима

In [ ]:
text_georgian = ''
text_georgian = ' '.join(df_georgian['reviews'].astype(str))

In [ ]:
STOPWORDS_RU = get_stop_words('russian')
wordcloud = WordCloud(
    width=800,  
    height=400, 
    background_color='white',  
    stopwords=STOPWORDS_RU,  
    max_words=200, 
    colormap='viridis',
    collocations=False 
).generate(text_georgian)


plt.figure(figsize=(12, 6)) 
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")  
plt.savefig('wordcloud_georgian.png')
plt.title('Облако слов из отзывов на грузинские рестораны')
plt.show()

In [ ]:
text_italian = ''
text_italian = ' '.join(df_italian['reviews'].astype(str))

In [ ]:
STOPWORDS_RU = get_stop_words('russian')
wordcloud = WordCloud(
    width=800,  
    height=400, 
    background_color='white', 
    stopwords=STOPWORDS_RU,  
    max_words=200,
    colormap='viridis',  
    collocations=False 
).generate(text_italian)

plt.figure(figsize=(12, 6))  
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")  
plt.savefig('wordcloud_italian.png')

plt.title('Облако слов из отзывов на итальянские рестораны')
plt.show()

Проанализировав облака слов, я заметила, что часто употребляются слова:"интерьер", "атмосфера", "уютное", "обслуживание". Кроме этого, много раз упоминаются
слова: хинкали, хачапури(наверное, это одни из самых любимых блюд посетителей), вкусная.
Наиболее часто встречющиеся слова в отзывах на итальяскую кухню: сервис, блюда, пицца, обслуживание, еда, вкусное, безумно. 


## Выводы 

Опишите в данном разделе, какие выводы вы можете сделать из вашей аналитики и визуализаций? (подробно)

1) Рестораны с высоким рейтингом максиум имеют около 5-10 лайков

2) Анализ подтверждает мысль, что качество не равно популярность. Рестораны с высоким рейтингом могут иметь малое количество лайков

3) Если у ресторанов лайков много, но сами они занимают в рейтинге не высокое место, 
то,скорее всего, маркетинг у таких заведений на высоком уровне. Быть может, они даже дают какие-то плюшки за оставленный отзыв. 
Замечание: в нашем случае кол-во лайков+ кол-во дизлайков = кол-во отзывов

4) Итальянских ресторанов с большим кол-вом лайков больше чем грузинских

5) Гистограмма рейтингов грузинских ресторанов смещена вправо (высокие оценки), у итальянских — более равномерное распределение.
Грузинские рестораны по сравнению с итальянскими выделяются своим стабильным высоким качеством

6) Вероятность того, что выбранное заведение грузинской кухни понравится больше,чем выбранное заведение итальянской кухни

7) В своем проекте я представила топ-10 ресторанов по разным критериям(рейтинг, кол-во лайков/дизлайков), поэтому в следующий раз я пойду в ресторан из предложенного мною списка 

8) Говоря об анализе слов, я поняла, что акцент в отзывах на грузинские рестораны делается на
обстановку и атмосферу, а в отзывах на итальяскую кухню на еду и эмоции


## Обсуждение

В этом разделе опишите:
- Что вы хотели сделать в рамках вашего исследования и что сделать удалось
- Что сделать не удалось и почему
- Как ваше исследование можно было бы улучшить
- Кому ваше исследование может быть полезно (или что можно сделать дополнительно, чтобы оно было полезным)

Вам не обязательно ставить планку научных работ. Например, если ваш датасет содержит информацию о 100 рецептов борща, а ваше исследование направлено на нахождение самого лучшего с точки зрения состава и пользовательских оценок, то оно банально может быть полезно людям, которые пытаются найти золотую середину в готовке этого самого борща (а если логику вашего исследования повторить с условными тефтелями, то оно ещё и переносимо на другие кейсы!)

В рамках моего исследования я хотела сравнить грузинские и итальянские рестораны, найти какие-то закономерности, 
а также выявить топ-10 ресторанов по различным критериям. Это удалось сделать.

Изначально я планировала брать данные с сайта Tripadvisor, однако меня он постоянно блокировал, а API у него платный. 
В Tripadvisor, конечно, больше информации о ресторанах, например, о ценовом сегменте. Еще там намного больше представлено отзывов.
В будущем хотелось бы собрать больше информации о ресторанах с сайтов, которые предоставляют такую информацию. (Irecommend в этом плане немного скудный). Также можно в будущем анализировать отзывы не только которые отображаются на главной странице, но и те, которые видны при переходе на вкладку 'Читать все отзывы'.

Мое исследование может быть полезно маркетологам ресторанов для разработки стратегии продвижения по привлечению новых клиентов, они могут увидеть рейтинг своего ресторана относительно других. Быть может, шеф-повара, благодаря облаку слов, смогут проанализировать часто встречащиеся блюда и сформировать меню в соответствии с результатами. Логику моего исследования можно повторить с любыми ресторанами сайта, других сайтов с ресторанами. Поменяв названия колонок, адаптиров их под новую тему, можно анализировать развлечения, отели и так далее.
